In [331]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

train = pd.read_csv('train.csv') #훈련 셋
test = pd.read_csv('test.csv') #정답 셋

In [332]:
train.head()

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,60 months,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B


In [333]:
test.head()

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
0,TEST_00000,16800000,36 months,8 years,MORTGAGE,132000000,19.64,12,주택 개선,0,394692,146604.0,0.0,0.0
1,TEST_00001,8400000,36 months,5 years,RENT,89971200,15.84,25,부채 통합,0,0,0.0,0.0,0.0
2,TEST_00002,17280000,36 months,6 years,RENT,150000000,8.41,20,신용 카드,0,1786980,281820.0,0.0,0.0
3,TEST_00003,14400000,36 months,5 years,MORTGAGE,66000000,13.72,30,신용 카드,1,669024,281724.0,0.0,0.0
4,TEST_00004,27600000,36 months,5 years,RENT,55200000,30.50,12,신용 카드,0,1250052,614844.0,0.0,0.0


In [334]:
train.shape

(96294, 15)

In [335]:
train.describe()

,대출금액,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
count,9.629400e+04,9.629400e+04,96294.000000,96294.000000,96294.000000,9.629400e+04,9.629400e+04,96294.000000,96294.000000
mean,1.830400e+07,9.392672e+07,19.379590,25.304827,0.345681,8.225035e+05,4.282282e+05,54.380584,0.005805
std,1.032908e+07,9.956871e+07,33.569559,12.088566,0.919119,1.027745e+06,4.402111e+05,1414.769218,0.079966
min,1.200000e+06,0.000000e+00,0.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,1.020000e+07,5.760000e+07,12.650000,17.000000,0.000000,3.075720e+05,1.346160e+05,0.000000,0.000000
50%,1.680000e+07,7.800000e+07,18.740000,24.000000,0.000000,5.976960e+05,2.870040e+05,0.000000,0.000000
75%,2.400000e+07,1.128000e+08,25.540000,32.000000,0.000000,1.055076e+06,5.702160e+05,0.000000,0.000000
max,4.200000e+07,1.080000e+10,9999.000000,169.000000,30.000000,4.195594e+07,5.653416e+06,75768.000000,4.000000


In [336]:
train['ID'].unique

<bound method Series.unique of 0        TRAIN_00000
1        TRAIN_00001
2        TRAIN_00002
3        TRAIN_00003
4        TRAIN_00004
            ...     
96289    TRAIN_96289
96290    TRAIN_96290
96291    TRAIN_96291
96292    TRAIN_96292
96293    TRAIN_96293
Name: ID, Length: 96294, dtype: object>

In [337]:
train['대출기간'].value_counts()

 36 months    64479
 60 months    31815
Name: 대출기간, dtype: int64

In [338]:
train['근로기간'].value_counts()

10+ years    31585
2 years       8450
< 1 year      7774
3 years       7581
1 year        6249
Unknown       5671
5 years       5665
4 years       5588
8 years       4888
6 years       3874
7 years       3814
9 years       3744
10+years       896
<1 year        370
3               89
1 years         56
Name: 근로기간, dtype: int64

In [339]:
df_train = train.copy()
df_test = test.copy()
df_train['대출기간'] = df_train['대출기간'].str.replace('[A-Za-z]+', '').astype(int)
df_train['근로기간'] = df_train['근로기간'].str.replace('[A-Za-z]+', '')
df_train['근로기간'] = df_train['근로기간'].str.replace(' ', '')
df_train['근로기간'] = df_train['근로기간'].replace(['<1'], '0')
df_train['근로기간'] = df_train['근로기간'].replace(['10+'], '10')
df_train['대출기간'] = df_train['대출기간'].replace([36], 3)
df_train['대출기간'] = df_train['대출기간'].replace([60], 5)

df_train.head()

/var/folders/dv/x61vgpd1541gc_b9zvr9c32m0000gn/T/ipykernel_67544/1305931257.py:3: FutureWarning:

The default value of regex will change from True to False in a future version.

/var/folders/dv/x61vgpd1541gc_b9zvr9c32m0000gn/T/ipykernel_67544/1305931257.py:4: FutureWarning:

The default value of regex will change from True to False in a future version.



,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,3,6,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,5,10,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,3,5,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,3,8,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,5,,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B


In [340]:
df_train.corr().iplot(kind='heatmap', colorscale='Greens')

/var/folders/dv/x61vgpd1541gc_b9zvr9c32m0000gn/T/ipykernel_67544/2608925664.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [341]:
df_train[(df_train['주택소유상태'] == 'MORTGAGE')]['대출등급'].value_counts()

B    14518
C    13106
A     9640
D     6163
E     3452
F      883
G      172
Name: 대출등급, dtype: int64

In [342]:
def mortgage_data(feature) :
    Mortgage = df_train[df_train['주택소유상태'] == 'MORTGAGE'][feature].value_counts()
    Credit = df_train[df_train['주택소유상태'] != 'MORTGAGE'][feature].value_counts()
    
    graph = pd.DataFrame([Mortgage, Credit])
    graph.index = ['Mortgage', 'Credit']

    return graph

In [343]:
mortgage_data('대출등급').iplot(kind='bar')

In [344]:
mortgage = df_train[df_train['주택소유상태'] == 'MORTGAGE']
credit = df_train[df_train['주택소유상태'] != 'MORTGAGE']

In [345]:
print(credit.shape)
print(mortgage.shape)

(48360, 15)
(47934, 15)


In [346]:
credit['대출등급'].value_counts()

C    14517
B    14299
D     7191
A     7132
E     3902
F     1071
G      248
Name: 대출등급, dtype: int64

In [347]:
credit.describe()

,대출금액,대출기간,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
count,4.836000e+04,48360.000000,4.836000e+04,48360.000000,48360.000000,48360.000000,4.836000e+04,4.836000e+04,48360.000000,48360.000000
mean,1.654197e+07,3.573077,8.138032e+07,19.317551,22.975455,0.302089,7.611690e+05,3.949530e+05,59.487097,0.004322
std,9.718082e+06,0.904297,6.766018e+07,9.342026,11.779978,0.881533,9.966323e+05,4.154145e+05,1476.400026,0.068679
min,1.200000e+06,3.000000,1.440000e+06,0.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,9.150000e+06,3.000000,4.896000e+07,12.570000,14.000000,0.000000,2.844240e+05,1.232400e+05,0.000000,0.000000
50%,1.440000e+07,3.000000,6.960000e+07,18.770000,21.000000,0.000000,5.552160e+05,2.625960e+05,0.000000,0.000000
75%,2.280000e+07,5.000000,9.600000e+07,25.720000,29.000000,0.000000,9.608280e+05,5.224560e+05,0.000000,0.000000
max,4.200000e+07,5.000000,5.576640e+09,672.520000,151.000000,30.000000,4.195594e+07,4.374576e+06,75768.000000,4.000000


In [348]:
def common_df(feature):
    A = credit[credit['대출등급']=='A'][feature].value_counts()
    B = credit[credit['대출등급']=='B'][feature].value_counts()
    C = credit[credit['대출등급']=='C'][feature].value_counts()
    D = credit[credit['대출등급']=='D'][feature].value_counts()
    E = credit[credit['대출등급']=='E'][feature].value_counts()
    F = credit[credit['대출등급']=='F'][feature].value_counts()
    G = credit[credit['대출등급']=='G'][feature].value_counts()
    
    df_graph = pd.DataFrame([A, B, C, D, E, F, G])
    df_graph.index = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
    
    return df_graph

In [359]:
df = common_df('대출목적')
df.iplot(kind='bar', barmode='stack')

In [349]:
df = common_df('대출기간')
df.iplot(kind='bar', barmode='stack')

In [350]:
high_fair = credit[credit['연간소득'] >= 1.440000e+07]['대출등급'].value_counts() #평균 대출금액 이상
low_fair = credit[credit['연간소득'] < 8.138032e+07]['대출등급'].value_counts()

df = pd.concat([high_fair, low_fair], axis=1, keys=['high_fair', 'low_fair'])
df.iplot(kind='bar', barmode='stack')

In [351]:
high_ratio = credit[credit['부채_대비_소득_비율'] >= 19.317551]['대출등급'].value_counts() #평균 대출 금액 이상
low_ratio = credit[credit['부채_대비_소득_비율'] < 19.317551]['대출등급'].value_counts()

df = pd.concat([high_ratio, low_ratio], axis=1, keys=['high', 'low'])
df.iplot(kind='bar', barmode='stack')

In [352]:
high_overdue = credit[credit['총연체금액'] >= 59]['대출등급'].value_counts() #연체 0번 초과
low_overdue = credit[credit['총연체금액'] < 59]['대출등급'].value_counts() #연체 0번 이하

df = pd.concat([high_overdue, low_overdue], axis=1, keys=['high', 'low'])
df.iplot(kind='bar', barmode='stack')

In [353]:
high_repay = credit[credit['총상환원금'] >= 5.552160e+05]['대출등급'].value_counts() #연체 0번 초과
low_repay = credit[credit['총상환원금'] < 5.552160e+05]['대출등급'].value_counts() 

df = pd.concat([high_repay, low_repay], axis=1, keys=['high', 'low'])
df.iplot(kind='bar', barmode='stack')

In [354]:
high_interest_repay = credit[credit['총상환이자'] >= 2.625960e+05]['대출등급'].value_counts() #연체 0번 초과
low_interest_repay = credit[credit['총상환이자'] < 2.625960e+05]['대출등급'].value_counts() 

df = pd.concat([high_interest_repay, low_interest_repay], axis=1, keys=['high', 'low'])
df.iplot(kind='bar', barmode='stack')

In [355]:
high_fair = credit[credit['연간소득'] >= 1.440000e+07]['대출등급'].value_counts() #평균 대출금액 이상
low_fair = credit[credit['연간소득'] < 8.138032e+07]['대출등급'].value_counts()

df1 = pd.concat([high_fair, low_fair], axis=1, keys=['high_fair', 'low_fair'])
df1.iplot(kind='bar', barmode='stack')

high_ratio = credit[credit['부채_대비_소득_비율'] >= 19.317551]['대출등급'].value_counts() #평균 대출 금액 이상
low_ratio = credit[credit['부채_대비_소득_비율'] < 19.317551]['대출등급'].value_counts()

df2 = pd.concat([high_ratio, low_ratio], axis=1, keys=['high_ratio', 'low_ratio'])
df2.iplot(kind='bar', barmode='stack')

high_overdue = credit[credit['총연체금액'] >= 59]['대출등급'].value_counts() #연체 0번 초과
low_overdue = credit[credit['총연체금액'] < 59]['대출등급'].value_counts() #연체 0번 이하

df3 = pd.concat([high_overdue, low_overdue], axis=1, keys=['high_overdue', 'low_overdue'])
df3.iplot(kind='bar', barmode='stack')

high_repay = credit[credit['총상환원금'] >= 5.552160e+05]['대출등급'].value_counts() #연체 0번 초과
low_repay = credit[credit['총상환원금'] < 5.552160e+05]['대출등급'].value_counts() 

df4 = pd.concat([high_repay, low_repay], axis=1, keys=['high_repay', 'low_repay'])
df4.iplot(kind='bar', barmode='stack')

high_interest_repay = credit[credit['총상환이자'] >= 2.625960e+05]['대출등급'].value_counts() #연체 0번 초과
low_interest_repay = credit[credit['총상환이자'] < 2.625960e+05]['대출등급'].value_counts() 

df5 = pd.concat([high_interest_repay, low_interest_repay], axis=1, keys=['high_interest_repay', 'low_interest_repay'])
df5.iplot(kind='bar', barmode='stack')


In [356]:
accounting = credit[credit['연간소득'] > 9.600000e+07]['대출등급'].value_counts()
period = credit[credit['대출금액']< 9.150000e+06]['대출등급'].value_counts()
total = credit['대출등급'].value_counts()

df = pd.concat([fair, period, total], axis=1, keys=['Fair', 'Period', 'Total'])
df

,Fair,Period,Total
B,3712,4147,14299
C,3088,4031,14517
A,2589,1569,7132
D,1352,1684,7191
E,804,536,3902
F,258,100,1071
G,53,21,248


In [254]:
cf.help()

Use 'cufflinks.help(figure)' to see the list of available parameters for the given figure.
Use 'DataFrame.iplot(kind=figure)' to plot the respective figure
Figures:
	bar
	box
	bubble
	bubble3d
	candle
	choroplet
	distplot
	heatmap
	histogram
	ohlc
	pie
	ratio
	scatter
	scatter3d
	scattergeo
	spread
	surface
	violin


In [322]:
mortgage.describe()

,대출금액,대출기간,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
count,4.793400e+04,47934.00000,4.793400e+04,47934.000000,47934.000000,47934.000000,4.793400e+04,4.793400e+04,47934.000000,47934.000000
mean,2.008168e+07,3.74928,1.065846e+08,19.442180,27.654900,0.389661,8.843831e+05,4.617993e+05,49.228689,0.007302
std,1.061871e+07,0.96807,1.223869e+08,46.645607,11.940276,0.953533,1.054610e+06,4.614659e+05,1349.737807,0.089906
min,1.200000e+06,3.00000,0.000000e+00,0.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,1.200000e+07,3.00000,6.600000e+07,12.740000,19.000000,0.000000,3.404670e+05,1.472760e+05,0.000000,0.000000
50%,1.800000e+07,3.00000,9.000000e+07,18.700000,26.000000,0.000000,6.601680e+05,3.147720e+05,0.000000,0.000000
75%,2.871000e+07,5.00000,1.248000e+08,25.360000,34.000000,0.000000,1.142529e+06,6.234150e+05,0.000000,0.000000
max,4.200000e+07,5.00000,1.080000e+10,9999.000000,169.000000,16.000000,3.555000e+07,5.653416e+06,75768.000000,2.000000


In [357]:
high_fair = mortgage[mortgage['연간소득'] >= 1.065846e+08]['대출등급'].value_counts() #평균 대출금액 이상
low_fair = mortgage[mortgage['연간소득'] < 1.065846e+08]['대출등급'].value_counts()

df1 = pd.concat([high_fair, low_fair], axis=1, keys=['high_fair', 'low_fair'])
df1.iplot(kind='bar', barmode='stack')

high_ratio = mortgage[mortgage['부채_대비_소득_비율'] >= 19.442180]['대출등급'].value_counts() #평균 대출 금액 이상
low_ratio = mortgage[mortgage['부채_대비_소득_비율'] < 19.442180]['대출등급'].value_counts()

df2 = pd.concat([high_ratio, low_ratio], axis=1, keys=['high_ratio', 'low_ratio'])
df2.iplot(kind='bar', barmode='stack')

high_overdue = mortgage[mortgage['총연체금액'] >= 49]['대출등급'].value_counts() #연체 0번 초과
low_overdue = mortgage[mortgage['총연체금액'] < 49]['대출등급'].value_counts() #연체 0번 이하

df3 = pd.concat([high_overdue, low_overdue], axis=1, keys=['high_overdue', 'low_overdue'])
df3.iplot(kind='bar', barmode='stack')

high_repay = mortgage[mortgage['총상환원금'] >= 6.601680e+05]['대출등급'].value_counts() #연체 0번 초과
low_repay = mortgage[mortgage['총상환원금'] < 6.601680e+05]['대출등급'].value_counts() 

df4 = pd.concat([high_repay, low_repay], axis=1, keys=['high_repay', 'low_repay'])
df4.iplot(kind='bar', barmode='stack')

high_interest_repay = mortgage[mortgage['총상환이자'] >= 3.147720e+05]['대출등급'].value_counts() #연체 0번 초과
low_interest_repay = mortgage[mortgage['총상환이자'] < 3.147720e+05]['대출등급'].value_counts() 

df5 = pd.concat([high_interest_repay, low_interest_repay], axis=1, keys=['high_interest_repay', 'low_interest_repay'])
df5.iplot(kind='bar', barmode='stack')

In [365]:
mortgage['대출목적'].unique()

array(['주택 개선', '부채 통합', '휴가', '소규모 사업', '신용 카드', '기타', '주요 구매', '자동차',
       '이사', '의료', '주택', '재생 에너지'], dtype=object)

In [381]:
loan_merge = mortgage[mortgage['대출목적']=='부채 통합']['대출등급'].value_counts()
creditcard = mortgage[mortgage['대출목적']=='신용 카드']['대출등급'].value_counts()
other = mortgage[(mortgage['대출목적']!='신용 카드') & (mortgage['대출목적']!='부채 통합')]['대출등급'].value_counts()

df = pd.concat([loan_merge, creditcard, other], axis=1, keys=['loan_merge', 'creditcard', 'other'])

df.iplot(kind='bar', barmode='stack')

In [382]:
loan_merge = credit[credit['대출목적']=='부채 통합']['대출등급'].value_counts()
creditcard = credit[credit['대출목적']=='신용 카드']['대출등급'].value_counts()
other = credit[(credit['대출목적']!='신용 카드') & (credit['대출목적']!='부채 통합')]['대출등급'].value_counts()

df = pd.concat([loan_merge, creditcard, other], axis=1, keys=['loan_merge', 'creditcard', 'other'])

df.iplot(kind='bar', barmode='stack')